In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import numpy as np
import speech_recognition as sr
import pyttsx3

engine = pyttsx3.init('sapi5')
my_voice = engine.getProperty('voices')
engine.setProperty('voice', my_voice[0].id)


def speak(audio):
    engine.say(audio)
    engine.runAndWait()


def takeCommand():
    recog = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening")
        recog.adjust_for_ambient_noise(source, duration=1)
        recog.pause_threshold = 1
        audio = recog.listen(source)

    try:
        print("Recognize")
        query = recog.recognize_google(audio, language='en-in')
        print(f"User said: {query}\n")

    except Exception as e:
        print(e)
        print("Sorry, I didn't get you.")
        return "None"
    return query


def sentimentAnalysis(s):
    # Here is TextBlob Sentiment Analysis
    sentiment = s
    blob = TextBlob(sentiment)
    textBlob_sentiment = blob.sentiment.polarity

    # Vader Sentiment Analysis
    vader = SentimentIntensityAnalyzer()
    vs = vader.polarity_scores(sentiment)
    vader_sentiment = vs["compound"]

    # Averaging out both of the sentiment scores
    average_sentiment = np.mean([textBlob_sentiment, vader_sentiment])
    print(vs)

    if average_sentiment > 0:
        return "Positive"
    elif average_sentiment < 0:
        return "Negative"
    else:
        return "Neutral"


if __name__ == "__main__":
    print("Please tell me a sentence on which you want to perform sentiment analysis.")
    speak("Please tell me a sentence on which you want to perform sentiment analysis.")
    s = takeCommand().lower()
    sentiment = sentimentAnalysis(s)
    speak(sentiment)
    print("Sentiment:", sentiment)

Please tell me a sentence on which you want to perform sentiment analysis.
Listening
Recognize
User said: I am feeling very bad

{'neg': 0.457, 'neu': 0.362, 'pos': 0.181, 'compound': -0.5095}
Sentiment: Negative


In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import numpy as np
import speech_recognition as sr
import pyttsx3
from langdetect import detect, LangDetectException

# Initialize text-to-speech engine
engine = pyttsx3.init('sapi5')
my_voice = engine.getProperty('voices')
engine.setProperty('voice', my_voice[0].id)

# Function to speak a given text
def speak(audio):
    engine.say(audio)
    engine.runAndWait()

# Function to take voice input from the user and recognize it initially in English
def takeCommand():
    recog = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening")
        recog.adjust_for_ambient_noise(source, duration=1)
        recog.pause_threshold = 1
        audio = recog.listen(source)

    try:
        print("Recognizing")
        # Recognize speech initially in English for language detection
        query = recog.recognize_google(audio, language='en')
        print(f"User said (initial): {query}\n")
    except Exception as e:
        print(e)
        print("Sorry, I didn't get you.")
        return "None", audio
    return query, audio

# Function to detect language from text
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Function to perform sentiment analysis using both TextBlob and VADER
def sentimentAnalysis(s):
    # TextBlob Sentiment Analysis
    blob = TextBlob(s)
    textBlob_sentiment = blob.sentiment.polarity

    # VADER Sentiment Analysis
    vader = SentimentIntensityAnalyzer()
    vs = vader.polarity_scores(s)
    vader_sentiment = vs["compound"]

    # Averaging out both of the sentiment scores
    average_sentiment = np.mean([textBlob_sentiment, vader_sentiment])
    print("VADER Sentiment Scores:", vs)

    if average_sentiment > 0:
        return "Positive"
    elif average_sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

# Main function
if __name__ == "__main__":
    print("Please tell me a sentence on which you want to perform sentiment analysis.")
    speak("Please tell me a sentence on which you want to perform sentiment analysis.")
    
    initial_text, audio = takeCommand()
    
    if initial_text == "None":
        speak("I did not understand what you said.")
    else:
        language = detect_language(initial_text)
        print(f"Detected language: {language}")

        # Map detected language to appropriate language code for Google Web Speech API
        language_codes = {
            'en': 'en-IN',
            'hi': 'hi-IN',
            'bn': 'bn-IN',
            'ta': 'ta-IN',
            'te': 'te-IN',
            'mr': 'mr-IN',
            'gu': 'gu-IN',
            'kn': 'kn-IN',
            'ml': 'ml-IN',
            'pa': 'pa-IN',
            'ur': 'ur-IN'
        }

        language_code = language_codes.get(language, 'ml-IN')

        recog = sr.Recognizer()
        try:
            print("Recognizing again with detected language")
            query = recog.recognize_google(audio, language=language_code)
            print(f"User said (final): {query}\n")
            speak(f"Detected language {language}")
            
            sentiment = sentimentAnalysis(query)
            speak(f"The sentiment of the sentence is {sentiment}")
            print("Sentiment:", sentiment)
        except Exception as e:
            print(e)
            speak("I did not understand what you said.")


Please tell me a sentence on which you want to perform sentiment analysis.
Listening
Recognizing
User said (initial): feeling very

Detected language: no
Recognizing again with detected language
User said (final): വെരി ബാഡ്

VADER Sentiment Scores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentiment: Neutral
